# Step Change Analysis During Alarm Episodes

This notebook analyzes operator step changes (OP and SP) taken during alarm episodes for target tags:
- **03LIC_1071** (Level Indicator Controller - TARGET)
- **03LIC_1016** (Level Indicator Controller)
- **03PIC_1013** (Pressure Indicator Controller)

## Approach
1. Extract unique alarm episodes from SSD data
2. For each alarm, define window: [`Tag_First_Transition_Start_minutes` (earliest), `AlarmEnd_rounded_minutes` + 60 minutes]
3. Filter CHANGE events occurring within these windows
4. Calculate Step = Value - PrevValue
5. Analyze step changes by PV ranges separately for OP and SP

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

# Define target tags
TARGET_TAGS = ['03LIC_1071', '03LIC_1016', '03PIC_1013']
print(f"Target tags for analysis: {TARGET_TAGS}")

Target tags for analysis: ['03LIC_1071', '03LIC_1016', '03PIC_1013']


## 1. Load Data

In [2]:
# Load SSD data (contains alarm episodes and tag deviation info)
ssd_df = pd.read_excel('DATA/SSD_1071_SSD_output_1071_7Jan2026.xlsx')
print(f"SSD data shape: {ssd_df.shape}")
print(f"SSD columns: {ssd_df.columns.tolist()[:10]}...")
print(f"Unique alarms: {ssd_df.groupby(['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes']).ngroups}")

SSD data shape: (10107, 31)
SSD columns: ['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes', 'TagName', 'Units', 'Tag_First_Transition_Start_minutes', 'Latest_Transition_Start_Time_minutes', 'Latest_Transition_End_Time_minutes', 'PV_at_latest_transition_start', 'PV_at_latest_transition_end', 'Max_Change_within_transition']...
Unique alarms: 609


In [3]:
# Load events data (contains operator CHANGE actions)
events_df = pd.read_csv('DATA/trip_filtered_events.csv', low_memory=False)
events_df['VT_Start'] = pd.to_datetime(events_df['VT_Start'])
print(f"Events data shape: {events_df.shape}")

# Filter for CHANGE events only
change_events_df = events_df[events_df['ConditionName'] == 'CHANGE'].copy()
print(f"CHANGE events shape: {change_events_df.shape}")
print(f"\nDescription value counts:")
print(change_events_df['Description'].value_counts().head(10))

Events data shape: (1600520, 29)
CHANGE events shape: (231714, 29)

Description value counts:
Description
OP          162020
SP           55104
MODE         11200
SO            1700
PVFL          1110
PVSOURCE       214
PVSRCOPT       158
OROPT           54
BYPASS          34
PV              28
Name: count, dtype: int64


In [4]:
# Load PV/OP time series data (for getting PV values at the time of events)
pv_op_df = pd.read_parquet('DATA/03LIC_1071_JAN_2026_filtered.parquet')
pv_op_df.index = pd.to_datetime(pv_op_df.index)
pv_op_df = pv_op_df.sort_index()
print(f"PV/OP data shape: {pv_op_df.shape}")
print(f"Date range: {pv_op_df.index.min()} to {pv_op_df.index.max()}")
print(f"\nTarget tag PV columns available:")
for tag in TARGET_TAGS:
    pv_col = f"{tag}.PV"
    if pv_col in pv_op_df.columns:
        print(f"  {pv_col}: {pv_op_df[pv_col].notna().sum()} non-null values")

PV/OP data shape: (1718039, 45)
Date range: 2022-01-03 22:45:00 to 2025-06-23 20:44:00

Target tag PV columns available:
  03LIC_1071.PV: 1718011 non-null values
  03LIC_1016.PV: 1718031 non-null values
  03PIC_1013.PV: 1718028 non-null values


## 2. Extract Alarm Windows from SSD Data

For each alarm episode:
- **Window Start**: Earliest `Tag_First_Transition_Start_minutes` across all tags for that alarm
- **Window End**: `AlarmEnd_rounded_minutes` + 60 minutes

In [5]:
# Convert SSD datetime columns to datetime
ssd_df['AlarmStart_rounded_minutes'] = pd.to_datetime(ssd_df['AlarmStart_rounded_minutes'])
ssd_df['AlarmEnd_rounded_minutes'] = pd.to_datetime(ssd_df['AlarmEnd_rounded_minutes'])
ssd_df['Tag_First_Transition_Start_minutes'] = pd.to_datetime(ssd_df['Tag_First_Transition_Start_minutes'])

# Group by alarm (AlarmStart, AlarmEnd) and get earliest transition start
alarm_windows = ssd_df.groupby(['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes']).agg(
    EarliestTransitionStart=('Tag_First_Transition_Start_minutes', 'min')
).reset_index()

# Calculate window end (AlarmEnd + 60 minutes)
alarm_windows['WindowStart'] = alarm_windows['EarliestTransitionStart']
alarm_windows['WindowEnd'] = alarm_windows['AlarmEnd_rounded_minutes'] + timedelta(minutes=60)

# Calculate window duration
alarm_windows['WindowDuration_minutes'] = (
    (alarm_windows['WindowEnd'] - alarm_windows['WindowStart']).dt.total_seconds() / 60
)

print(f"Total alarm episodes: {len(alarm_windows)}")
print(f"\nWindow duration statistics (minutes):")
print(alarm_windows['WindowDuration_minutes'].describe())
alarm_windows.head()

Total alarm episodes: 609

Window duration statistics (minutes):
count    609.000000
mean     167.208539
std       42.552029
min       99.000000
25%      149.000000
50%      152.000000
75%      168.000000
max      502.000000
Name: WindowDuration_minutes, dtype: float64


,AlarmStart_rounded_minutes,AlarmEnd_rounded_minutes,EarliestTransitionStart,WindowStart,WindowEnd,WindowDuration_minutes
0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 07:27:00,2022-01-05 10:33:00,186.0
1,2022-01-07 09:55:00,2022-01-07 10:00:00,2022-01-07 08:29:00,2022-01-07 08:29:00,2022-01-07 11:00:00,151.0
2,2022-01-07 13:33:00,2022-01-07 13:36:00,2022-01-07 12:07:00,2022-01-07 12:07:00,2022-01-07 14:36:00,149.0
3,2022-01-07 14:17:00,2022-01-07 14:19:00,2022-01-07 12:51:00,2022-01-07 12:51:00,2022-01-07 15:19:00,148.0
4,2022-01-07 14:54:00,2022-01-07 14:58:00,2022-01-07 13:28:00,2022-01-07 13:28:00,2022-01-07 15:58:00,150.0


## 3. Filter CHANGE Events Within Alarm Windows

Extract operator CHANGE actions (OP and SP) for target tags that occurred during the alarm windows.

In [6]:
# Filter CHANGE events for target tags
target_changes = change_events_df[change_events_df['Source'].isin(TARGET_TAGS)].copy()
print(f"CHANGE events for target tags: {len(target_changes)}")
print(f"\nBy tag:")
print(target_changes['Source'].value_counts())
print(f"\nBy Description (OP vs SP):")
print(target_changes['Description'].value_counts())

CHANGE events for target tags: 20504

By tag:
Source
03PIC_1013    15568
03LIC_1071     3076
03LIC_1016     1860
Name: count, dtype: int64

By Description (OP vs SP):
Description
OP          17852
SP           1566
MODE         1078
PVSRCOPT        4
PVSOURCE        4
Name: count, dtype: int64


In [7]:
# Filter CHANGE events that fall within alarm windows
def get_events_within_windows(events_df, alarm_windows):
    """Filter events that fall within any alarm window."""
    events_in_windows = []
    
    for idx, row in alarm_windows.iterrows():
        window_start = row['WindowStart']
        window_end = row['WindowEnd']
        alarm_start = row['AlarmStart_rounded_minutes']
        alarm_end = row['AlarmEnd_rounded_minutes']
        
        # Find events within this window
        mask = (events_df['VT_Start'] >= window_start) & (events_df['VT_Start'] <= window_end)
        window_events = events_df[mask].copy()
        
        if len(window_events) > 0:
            window_events['AlarmStart'] = alarm_start
            window_events['AlarmEnd'] = alarm_end
            window_events['WindowStart'] = window_start
            window_events['WindowEnd'] = window_end
            window_events['AlarmEpisodeID'] = idx
            events_in_windows.append(window_events)
    
    if events_in_windows:
        return pd.concat(events_in_windows, ignore_index=True)
    return pd.DataFrame()

# Get target tag events within alarm windows
events_in_alarm_windows = get_events_within_windows(target_changes, alarm_windows)
print(f"Total operator actions during alarm windows: {len(events_in_alarm_windows)}")
print(f"\nBy tag:")
print(events_in_alarm_windows['Source'].value_counts())
print(f"\nBy Description (OP vs SP):")
print(events_in_alarm_windows['Description'].value_counts())

Total operator actions during alarm windows: 11468

By tag:
Source
03PIC_1013    6610
03LIC_1071    3524
03LIC_1016    1334
Name: count, dtype: int64

By Description (OP vs SP):
Description
OP      9752
SP      1196
MODE     520
Name: count, dtype: int64


## 4. Calculate Step Changes

Calculate `Step = Value - PrevValue` and handle NaN values.

In [8]:
# Convert Value and PrevValue to numeric
events_in_alarm_windows['Value'] = pd.to_numeric(events_in_alarm_windows['Value'], errors='coerce')
events_in_alarm_windows['PrevValue'] = pd.to_numeric(events_in_alarm_windows['PrevValue'], errors='coerce')

# Calculate Step Change
events_in_alarm_windows['Step_Change'] = events_in_alarm_windows['Value'] - events_in_alarm_windows['PrevValue']

# Statistics on NaN values
total_actions = len(events_in_alarm_windows)
nan_value = events_in_alarm_windows['Value'].isna().sum()
nan_prev_value = events_in_alarm_windows['PrevValue'].isna().sum()
nan_step = events_in_alarm_windows['Step_Change'].isna().sum()

print("=" * 60)
print("NaN VALUE STATISTICS")
print("=" * 60)
print(f"Total operator actions during alarm windows: {total_actions}")
print(f"\nNaN counts:")
print(f"  Value is NaN:       {nan_value} ({nan_value/total_actions*100:.2f}%)")
print(f"  PrevValue is NaN:   {nan_prev_value} ({nan_prev_value/total_actions*100:.2f}%)")
print(f"  Step_Change is NaN: {nan_step} ({nan_step/total_actions*100:.2f}%)")
print(f"\nRows to be used (non-NaN Step_Change): {total_actions - nan_step}")
print("=" * 60)

# Break down by tag and description
print("\nNaN Step_Change breakdown by Tag and Description:")
nan_breakdown = events_in_alarm_windows.groupby(['Source', 'Description']).agg(
    Total=('Step_Change', 'size'),
    NaN_Count=('Step_Change', lambda x: x.isna().sum())
).reset_index()
nan_breakdown['NaN_%'] = (nan_breakdown['NaN_Count'] / nan_breakdown['Total'] * 100).round(2)
print(nan_breakdown.to_string(index=False))

NaN VALUE STATISTICS
Total operator actions during alarm windows: 11468

NaN counts:
  Value is NaN:       806 (7.03%)
  PrevValue is NaN:   6135 (53.50%)
  Step_Change is NaN: 6187 (53.95%)

Rows to be used (non-NaN Step_Change): 5281

NaN Step_Change breakdown by Tag and Description:
    Source Description  Total  NaN_Count  NaN_%
03LIC_1016        MODE     98         98 100.00
03LIC_1016          OP    826        554  67.07
03LIC_1016          SP    410        205  50.00
03LIC_1071        MODE    192        192 100.00
03LIC_1071          OP   2546       1325  52.04
03LIC_1071          SP    786        393  50.00
03PIC_1013        MODE    230        230 100.00
03PIC_1013          OP   6380       3190  50.00


In [9]:
# Filter to valid step changes (non-NaN)
valid_step_changes = events_in_alarm_windows[events_in_alarm_windows['Step_Change'].notna()].copy()
print(f"Valid step changes for analysis: {len(valid_step_changes)}")
print(f"\nBy tag:")
print(valid_step_changes['Source'].value_counts())
print(f"\nBy Description (OP vs SP):")
print(valid_step_changes['Description'].value_counts())

Valid step changes for analysis: 5281

By tag:
Source
03PIC_1013    3190
03LIC_1071    1614
03LIC_1016     477
Name: count, dtype: int64

By Description (OP vs SP):
Description
OP    4683
SP     598
Name: count, dtype: int64


## 5. Merge with PV Data to Get PV Ranges

Get the PV value at the time of each operator action to create PV ranges for analysis.

In [10]:
# Function to get PV value at the time of event using nearest timestamp match
def get_pv_at_event_time(events_df, pv_op_df, tag):
    """Get PV value at the time of each event for a given tag."""
    pv_col = f"{tag}.PV"
    
    if pv_col not in pv_op_df.columns:
        print(f"Warning: {pv_col} not found in PV/OP data")
        return None
    
    # Get events for this tag
    tag_events = events_df[events_df['Source'] == tag].copy()
    
    if len(tag_events) == 0:
        return None
    
    # Round event times to nearest minute for matching
    tag_events['EventTime_rounded'] = tag_events['VT_Start'].dt.floor('min')
    
    # Get PV values at event times using merge_asof for nearest match
    pv_series = pv_op_df[[pv_col]].copy()
    pv_series = pv_series.reset_index()
    pv_series.columns = ['TimeStamp', 'PV_at_event']
    
    tag_events = tag_events.sort_values('VT_Start')
    pv_series = pv_series.sort_values('TimeStamp')
    
    # Merge using nearest timestamp within 5 minutes tolerance
    merged = pd.merge_asof(
        tag_events, 
        pv_series, 
        left_on='VT_Start', 
        right_on='TimeStamp',
        direction='nearest',
        tolerance=pd.Timedelta('5min')
    )
    
    return merged

# Get PV values at event times for each target tag
result_dfs = []
for tag in TARGET_TAGS:
    merged = get_pv_at_event_time(valid_step_changes, pv_op_df, tag)
    if merged is not None and len(merged) > 0:
        merged['TagName'] = tag
        result_dfs.append(merged)
        print(f"{tag}: {len(merged)} events merged with PV data")
        print(f"  PV range: {merged['PV_at_event'].min():.2f} to {merged['PV_at_event'].max():.2f}")
        print(f"  PV null count: {merged['PV_at_event'].isna().sum()}")

# Combine all results
if result_dfs:
    step_changes_with_pv = pd.concat(result_dfs, ignore_index=True)
    print(f"\nTotal events with PV: {len(step_changes_with_pv)}")

03LIC_1071: 1614 events merged with PV data
  PV range: -1.33 to 103.13
  PV null count: 20
03LIC_1016: 477 events merged with PV data
  PV range: 0.19 to 99.71
  PV null count: 0
03PIC_1013: 3190 events merged with PV data
  PV range: 19.70 to 318.26
  PV null count: 0

Total events with PV: 5281


In [11]:
# Filter out events where PV couldn't be matched
step_changes_with_pv = step_changes_with_pv[step_changes_with_pv['PV_at_event'].notna()].copy()
print(f"Events with valid PV values: {len(step_changes_with_pv)}")

# Check PV statistics for each tag
print("\nPV Statistics at Event Times:")
for tag in TARGET_TAGS:
    tag_data = step_changes_with_pv[step_changes_with_pv['TagName'] == tag]['PV_at_event']
    if len(tag_data) > 0:
        print(f"\n{tag}:")
        print(f"  Count: {len(tag_data)}")
        print(f"  Mean: {tag_data.mean():.2f}")
        print(f"  Std: {tag_data.std():.2f}")
        print(f"  Min: {tag_data.min():.2f}")
        print(f"  Max: {tag_data.max():.2f}")
        print(f"  Q25: {tag_data.quantile(0.25):.2f}")
        print(f"  Q50: {tag_data.quantile(0.50):.2f}")
        print(f"  Q75: {tag_data.quantile(0.75):.2f}")

Events with valid PV values: 5261

PV Statistics at Event Times:

03LIC_1071:
  Count: 1594
  Mean: 41.46
  Std: 26.13
  Min: -1.33
  Max: 103.13
  Q25: 25.03
  Q50: 41.59
  Q75: 60.30

03LIC_1016:
  Count: 477
  Mean: 32.99
  Std: 25.92
  Min: 0.19
  Max: 99.71
  Q25: 8.53
  Q50: 31.00
  Q75: 51.20

03PIC_1013:
  Count: 3190
  Mean: 302.78
  Std: 45.51
  Min: 19.70
  Max: 318.26
  Q25: 318.09
  Q50: 318.18
  Q75: 318.21


## 6. Step Change Analysis by PV Ranges

Create PV ranges using quantile-based binning (10 bins) and analyze step changes separately for OP and SP.

In [12]:
def analyze_step_changes_by_pv_range(data, tag, action_type, n_bins=10):
    """
    Analyze step changes by PV ranges for a specific tag and action type (OP/SP).
    
    Parameters:
    -----------
    data : DataFrame
        Data with Step_Change and PV_at_event columns
    tag : str
        Target tag name
    action_type : str
        'OP' or 'SP'
    n_bins : int
        Number of PV range bins
        
    Returns:
    --------
    tuple : (step_stats_df, directional_df)
    """
    # Filter for tag and action type
    mask = (data['TagName'] == tag) & (data['Description'] == action_type)
    tag_data = data[mask].copy()
    
    if len(tag_data) < 5:
        print(f"Insufficient data for {tag} ({action_type}): {len(tag_data)} records")
        return None, None
    
    # Create PV range bins using quantiles
    try:
        tag_data['PV_Range'] = pd.qcut(tag_data['PV_at_event'], q=n_bins, duplicates='drop')
    except ValueError:
        # If not enough unique values for n_bins, use fewer bins
        tag_data['PV_Range'] = pd.cut(tag_data['PV_at_event'], bins=min(n_bins, len(tag_data.PV_at_event.unique())))
    
    # Calculate step change statistics by PV range
    step_stats = tag_data.groupby('PV_Range', observed=True).agg(
        Data_Points=('Step_Change', 'size'),
        Step_Min=('Step_Change', 'min'),
        Step_Max=('Step_Change', 'max'),
        Step_Mean=('Step_Change', 'mean'),
        Step_Median=('Step_Change', 'median'),
        Step_Std=('Step_Change', 'std'),
        Step_25th=('Step_Change', lambda x: x.quantile(0.25)),
        Step_75th=('Step_Change', lambda x: x.quantile(0.75))
    ).reset_index()
    
    # Directional breakdown
    tag_data['Step_Direction'] = tag_data['Step_Change'].apply(
        lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Zero')
    )
    
    directional = tag_data.groupby(['PV_Range', 'Step_Direction'], observed=True).size().unstack(fill_value=0)
    directional = directional.reset_index()
    
    # Add percentages
    for col in ['Positive', 'Negative', 'Zero']:
        if col not in directional.columns:
            directional[col] = 0
    
    directional['Total'] = directional['Positive'] + directional['Negative'] + directional['Zero']
    directional['Positive_%'] = (directional['Positive'] / directional['Total'] * 100).round(2)
    directional['Negative_%'] = (directional['Negative'] / directional['Total'] * 100).round(2)
    directional['Zero_%'] = (directional['Zero'] / directional['Total'] * 100).round(2)
    
    return step_stats, directional


def display_analysis(step_stats, directional, tag, action_type):
    """Display step change analysis results."""
    print(f"\n{'='*80}")
    print(f"TAG: {tag} — {action_type} Action Step Changes by PV Ranges")
    print(f"{'='*80}")
    
    if step_stats is not None:
        display_cols = ['PV_Range', 'Data_Points', 'Step_Min', 'Step_Max', 
                       'Step_Mean', 'Step_Median', 'Step_Std', 'Step_25th', 'Step_75th']
        print("\nStep Change Statistics:")
        print(step_stats[display_cols].to_string(index=False))
        
    if directional is not None:
        print(f"\n{action_type} Actions — Directional Breakdown:")
        display_cols = ['PV_Range', 'Total', 'Positive', 'Negative', 'Zero',
                       'Positive_%', 'Negative_%', 'Zero_%']
        available_cols = [c for c in display_cols if c in directional.columns]
        print(directional[available_cols].to_string(index=False))

---
## Tag-wise Analysis

Analyze step changes for each target tag separately for OP and SP actions.

In [13]:
# Run analysis for all target tags - OP Actions
print("="*100)
print("OP ACTION ANALYSIS")
print("="*100)

all_op_results = {}
for tag in TARGET_TAGS:
    step_stats, directional = analyze_step_changes_by_pv_range(step_changes_with_pv, tag, 'OP', n_bins=10)
    if step_stats is not None:
        all_op_results[tag] = {'step_stats': step_stats, 'directional': directional}
        display_analysis(step_stats, directional, tag, 'OP')

OP ACTION ANALYSIS

TAG: 03LIC_1071 — OP Action Step Changes by PV Ranges

Step Change Statistics:
         PV_Range  Data_Points  Step_Min  Step_Max  Step_Mean  Step_Median  Step_Std  Step_25th  Step_75th
 (-1.329, -0.228]          121   -2.0000   15.0000   2.099174          2.0  2.554098        2.0        2.0
  (-0.228, 9.072]          121  -27.0000   50.0000   2.020661          2.0  7.630639        1.0        2.0
   (9.072, 27.06]          126  -38.7079   35.0000   0.751983          0.1  7.731267       -2.0        2.0
  (27.06, 37.868]          122  -49.7910    7.0000   0.182068          1.0  5.543571       -0.1        2.0
  (37.868, 41.87]          112   -9.3038    5.0000  -1.224989         -0.1  2.189577       -2.0       -0.1
  (41.87, 49.429]          139  -20.0000   10.0000   0.067892          0.1  3.552920       -2.0        2.0
 (49.429, 57.338]          101  -22.0971   11.8083  -0.892740         -2.0  3.638466       -2.0        1.0
  (57.338, 64.34]          124  -31.0369   10

In [14]:
# Run analysis for all target tags - SP Actions
print("\n\n")
print("="*100)
print("SP ACTION ANALYSIS")
print("="*100)

all_sp_results = {}
for tag in TARGET_TAGS:
    step_stats, directional = analyze_step_changes_by_pv_range(step_changes_with_pv, tag, 'SP', n_bins=10)
    if step_stats is not None:
        all_sp_results[tag] = {'step_stats': step_stats, 'directional': directional}
        display_analysis(step_stats, directional, tag, 'SP')




SP ACTION ANALYSIS

TAG: 03LIC_1071 — SP Action Step Changes by PV Ranges

Step Change Statistics:
                     PV_Range  Data_Points  Step_Min  Step_Max  Step_Mean  Step_Median  Step_Std  Step_25th  Step_75th
(-1.2999999999999998, 23.438]           40  -10.0000   35.0000   3.553177      2.00000  7.865789     1.0000   5.000000
             (23.438, 28.185]           57   -2.0000   20.4648   1.354009      1.00000  3.083906     0.0100   2.000000
              (28.185, 29.47]           22   -1.0000    3.0000   1.772727      2.00000  0.869144     1.2500   2.000000
              (29.47, 33.828]           38   -8.0000   10.3378   1.143992      1.89355  2.805217     1.0000   2.000000
             (33.828, 37.548]           40   -2.0000    3.0000   0.683283      0.01000  1.287865     0.0100   2.000000
             (37.548, 43.776]           39  -17.4591    5.0000  -1.468764      0.30000  5.142461    -2.0000   1.000000
             (43.776, 47.642]           39   -2.0000    3.0000   

## Summary Statistics

Overall summary of operator actions during alarm episodes.

In [15]:
# Summary of all operator actions during alarm episodes
print("="*100)
print("SUMMARY: Operator Actions During Alarm Episodes")
print("="*100)

summary_data = []
for tag in TARGET_TAGS:
    for action_type in ['OP', 'SP']:
        mask = (step_changes_with_pv['TagName'] == tag) & (step_changes_with_pv['Description'] == action_type)
        tag_data = step_changes_with_pv[mask]
        
        if len(tag_data) > 0:
            summary_data.append({
                'TagName': tag,
                'Action_Type': action_type,
                'Count': len(tag_data),
                'Step_Mean': tag_data['Step_Change'].mean(),
                'Step_Std': tag_data['Step_Change'].std(),
                'Step_Min': tag_data['Step_Change'].min(),
                'Step_Max': tag_data['Step_Change'].max(),
                'Positive_Steps': (tag_data['Step_Change'] > 0).sum(),
                'Negative_Steps': (tag_data['Step_Change'] < 0).sum(),
                'Zero_Steps': (tag_data['Step_Change'] == 0).sum()
            })

summary_df = pd.DataFrame(summary_data)
print("\nAction counts by Tag and Type:")
print(summary_df.to_string(index=False))

# Total alarms analyzed
print(f"\n\nTotal alarm episodes analyzed: {len(alarm_windows)}")
print(f"Alarm episodes with operator actions: {events_in_alarm_windows['AlarmEpisodeID'].nunique()}")

SUMMARY: Operator Actions During Alarm Episodes

Action counts by Tag and Type:
   TagName Action_Type  Count  Step_Mean  Step_Std  Step_Min  Step_Max  Positive_Steps  Negative_Steps  Zero_Steps
03LIC_1071          OP   1201  -0.212221  6.647277  -50.0000   50.0000             607             594           0
03LIC_1071          SP    393  -0.035189  5.315226  -49.3290   35.0000             254             139           0
03LIC_1016          OP    272  -0.443544  9.410404  -72.7805   30.0000             164             108           0
03LIC_1016          SP    205  -0.491271  6.570198  -44.8531   16.0464             123              82           0
03PIC_1013          OP   3190  -0.112146  1.148288   -4.0000    4.0000            1294            1889           7


Total alarm episodes analyzed: 609
Alarm episodes with operator actions: 243
